In [1]:
from tensorflow.keras.applications import MobileNet
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.mobilenet import preprocess_input
import numpy as np
import os
from sklearn.decomposition import PCA

img_height = 224  # Replace with your desired height
img_width = 224   # Replace with your desired width

# Load pre-trained MobileNet model without top classification layers
base_model = MobileNet(weights='imagenet', include_top=False, input_shape=(img_height, img_width, 3))

# Extract features using MobileNet
def extract_features_mobilenet(img_path):
    img = image.load_img(img_path, target_size=(img_height, img_width))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    features = base_model.predict(x)
    return features

# Get features and labels for your dataset
dataset_directory = 'C:/Users/saisr/OneDrive/Desktop/HAR_stftimg'
feature_list = []
labels = []

for directory in os.listdir(dataset_directory):
    dir_path = os.path.join(dataset_directory, directory)
    if os.path.isdir(dir_path):
        for img_file in os.listdir(dir_path):
            img_path = os.path.join(dir_path, img_file)
            features = extract_features_mobilenet(img_path)
            feature_list.append(features)
            labels.append(directory)  # Assuming directory name is the class label

# Concatenate features and convert labels to numeric format
concatenated_features = np.concatenate(feature_list, axis=0)
label_dict = {label: index for index, label in enumerate(np.unique(labels))}
numeric_labels = [label_dict[label] for label in labels]

# Flatten the extracted features
flatten_features_mobilenet = concatenated_features.reshape(concatenated_features.shape[0], -1)

# Perform PCA for dimensionality reduction
pca = PCA(n_components=200)  # Adjust the number of components as needed
reduced_features_mobilenet = pca.fit_transform(flatten_features_mobilenet)

# Use reduced_features_mobilenet for further analysis or model training


1/1 [==============================] - 0s 91ms/step


1/1 [==============================] - 0s 70ms/step


1/1 [==============================] - 0s 81ms/step


1/1 [==============================] - 0s 142ms/step


1/1 [==============================] - 0s 81ms/step


...

1/1 [==============================] - 0s 293ms/step


1/1 [==============================] - 0s 64ms/step


1/1 [==============================] - 0s 179ms/step


1/1 [==============================] - 0s 64ms/step


1/1 [==============================] - 0s 83ms/step


In [4]:
concatenated_features.shape

(6450, 7, 7, 1024)

In [2]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
import numpy as np
from sklearn.metrics import confusion_matrix

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(reduced_features_mobilenet, numeric_labels, test_size=0.1, random_state=42)

X_train = np.array(X_train)
y_train = np.array(y_train)
X_test = np.array(X_test)
y_test = np.array(y_test)

# Define EarlyStopping callback
early_stopping = EarlyStopping(monitor='val_accuracy', patience=10, restore_best_weights=True)

# Define a simple fully connected neural network
model = Sequential()
model.add(Dense(512, activation='relu', input_shape=(reduced_features_mobilenet.shape[1],)))
model.add(Dropout(0.5))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(np.unique(labels)), activation='softmax'))  # Output layer with number of classes

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model with EarlyStopping callback
history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test), callbacks=[early_stopping])

# Generate predictions for the test set
predictions = model.predict(X_test)
predicted_classes = np.argmax(predictions, axis=1)

# Create confusion matrix
confusion_mtx = confusion_matrix(y_test, predicted_classes)

# Print the confusion matrix
print("Confusion Matrix:")
print(confusion_mtx)

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print("Test Accuracy:", accuracy)


Epoch 1/50
182/182 [==============================] - 4s 7ms/step - loss: 3.0271 - accuracy: 0.5187 - val_loss: 0.7911 - val_accuracy: 0.6000
Epoch 2/50
182/182 [==============================] - 2s 10ms/step - loss: 0.9730 - accuracy: 0.5752 - val_loss: 0.7680 - val_accuracy: 0.6233
Epoch 3/50
182/182 [==============================] - 2s 10ms/step - loss: 0.8271 - accuracy: 0.5900 - val_loss: 0.7445 - val_accuracy: 0.5969
Epoch 4/50
182/182 [==============================] - 1s 7ms/step - loss: 0.7712 - accuracy: 0.6133 - val_loss: 0.7365 - val_accuracy: 0.6434
Epoch 5/50
182/182 [==============================] - 1s 6ms/step - loss: 0.7309 - accuracy: 0.6312 - val_loss: 0.7277 - val_accuracy: 0.6093
Epoch 6/50
182/182 [==============================] - 1s 7ms/step - loss: 0.7211 - accuracy: 0.6381 - val_loss: 0.7114 - val_accuracy: 0.6651
Epoch 7/50
182/182 [==============================] - 1s 5ms/step - loss: 0.6984 - accuracy: 0.6474 - val_loss: 0.7078 - val_accuracy: 0.6403
Epoc